# CREDIT SCORE PREDICTION NOTEBOOK

## BUSINESS UNDERSTANDING

    In this project, the primary objective is to develop a robust and accurate model that evaluates an individual's creditworthiness. The project involves leveraging machine learning algorithms and statistical analyses to assess a variety of relevant factors, such as financial history, payment behavior, outstanding debts, and other relevant variables. By understanding the intricate relationships between these factors, the predictive model aims to generate a numerical credit score that serves as a reliable indicator of an individual's likelihood to meet their financial obligations. The business value lies in providing financial institutions with an efficient and objective tool for risk assessment, aiding in more informed decision-making during the lending process. This predictive model not only enhances the efficiency of credit evaluations but also contributes to minimizing default risks and optimizing the allocation of resources within the lending ecosystem.







## DATA UNDERSTANDING

The data used in this project was extracted from kaggle.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
df = pd.read_csv('Credit-Score-Prediction/loan.csv')

### Data Preprocessing

The purpose of data preprocessing is to:
- Check missing values

- Duplicates

- display total number of rows

In [1]:
# In this cell we will create data preprocessing function

class DataInfo():

    """Class for processing and giving the general information of the data"""
    def __init__(self, data):
        # Initializing the primary elements
        self.data = data

    def data_preprocessing(self):
        # Function for checking the missing values
        self.missing_values = self.data.isnull().sum().sum()
        print("Missing Values:")
        print(self.missing_values)
        print()

        # Check for duplicates
        self.duplicates = self.data.duplicated().sum()
        print("Number of Duplicates:")
        print(self.duplicates)
        print()

        # Display total number of rows
        self.num_rows = self.data.shape[0]
        print("Total Number of Rows:", self.num_rows)
        print()

         # Statistical summary
        print("Statistical Summary:")
        print(self.data.describe())


    def fit_transform(self):
        # Function for applying the data_preprocessing function above
        
        return self.data_preprocessing()